# Experimento mestrado 1
## Executando na pasta principal

In [1]:
import os
os.chdir('../../../') #Executando na pasta Visual_Memory
import sys
sys.path.append('./include')
sys.path.append('./src')

## Copilando arquivos para teste

In [2]:
if os.system("cd Workbench/ && ./convertendo_all.sh && cd ..") != 0:
    print "Erro de copilação"
    sys.exit(0)

## Criando configurações padroes e arquivo de execução

In [3]:
text = open("./Data/config.ini", "w")
text.write("[SETTINGS - VISUAL MEMORY]\nnumber_robots = 3")
text.close()

text = open("./run.sh", "w")
text.write("#!/bin/bash\n\npython visualMemory.py")
text.close()

## Executando VisualMemory e removendo arquivo de execução

In [4]:
os.system("chmod 777 run.sh")
os.system("screen -d -m -S 'VisualMemory' './run.sh'")
os.system("gnome-terminal -x sh -c 'screen -R VisualMemory'")
os.system("rm run.sh")

0

In [5]:
from Blackboard import *

In [6]:
bkb = Blackboard( )

In [21]:
bkb.write_float('VISION_RB01_TAG', 3)

In [22]:
bkb.read_float('VISION_RB01_TAG')

0.0